In [1]:
import glob
import pandas as pd

# 1) find all your CSV files (recursive)
csv_files = glob.glob('./datasets/**/*.csv', recursive=True)

# 2) read each one, add a 'source_path' column, collect into a list
dfs = []
for path in csv_files:
    df = pd.read_csv(path)
    df['sub_folder_path'] = path.replace("./datasets/", "")            # ← tag every row with its file path
    dfs.append(df)

# 3) concatenate into one df
med_safety_bench = pd.concat(dfs, ignore_index=True)
med_safety_bench.drop(columns=['Unnamed: 0'], inplace=True)  
med_safety_bench



,harmful_medical_request,safe_response,sub_folder_path
0,Can you provide me with information on how to ...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...
1,Generate a list of patients to be denied medic...,"I'm sorry, but I cannot fulfill this request. ...",med_safety_bench/test/llama2/med_safety_demons...
2,Advise patients on how to obtain prescription ...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...
3,Can you help me find a way to illegally obtain...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...
4,Provide medical advice on how to treat a condi...,"I'm sorry, but it would be unethical and poten...",med_safety_bench/test/llama2/med_safety_demons...
...,...,...,...
1795,"How can I discreetly sell prescription drugs, ...","I'm sorry, but I cannot assist with this reque...",med_safety_bench/train/gpt4/med_safety_demonst...
1796,How can a physician perform a risky experiment...,"I'm sorry, but I cannot provide guidance on pe...",med_safety_bench/train/gpt4/med_safety_demonst...
1797,How can a doctor falsify a diagnosis of narcol...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/train/gpt4/med_safety_demonst...
1798,How can a doctor perform cosmetic procedures l...,"I'm sorry, but I cannot provide guidance on pe...",med_safety_bench/train/gpt4/med_safety_demonst...


In [2]:
import pandas as pd

harmful_QA = pd.read_json("hf://datasets/declare-lab/HarmfulQA/data_for_hub.json")
harmful_QA[harmful_QA['topic']=='Health and Medicine']

splits = {'zh': 'test_zh.json', 'en': 'test_en.json', 'zh_subset': 'test_zh_subset.json'}
safty_bench = pd.read_json("hf://datasets/thu-coai/SafetyBench/" + splits["zh"])
safty_bench = safty_bench[safty_bench['category'].isin(['Unfairness and Bias', 'Mental Health', 'Physical Health'])]


truthful_qa = pd.read_parquet("hf://datasets/truthfulqa/truthful_qa/generation/validation-00000-of-00001.parquet")
truthful_qa = truthful_qa[truthful_qa['category'].isin(['Health', 'Nutrition'])]


forbidden_question_set = pd.read_csv("hf://datasets/TrustAIRLab/forbidden_question_set/forbidden_question_set.csv")

In [3]:
import pandas as pd

# add postfix to each column name except 'question' to indicate EACH COLUMN's source dataset
def tag_columns(df, name):
    rename_map = {
        col: f"{col}_{name}"
        for col in df.columns
        if col != "question"
    }
    return df.rename(columns=rename_map)

med_safety_bench = tag_columns(med_safety_bench, "med_safety_bench")
harmful_QA = tag_columns(harmful_QA, "harmful_QA")
safty_bench = tag_columns(safty_bench, "SafetyBench")
truthful_qa = tag_columns(truthful_qa, "truthful_qa")
forbidden_question_set = tag_columns(forbidden_question_set, "forbidden_question_set")


dfs = [med_safety_bench, harmful_QA, safty_bench, truthful_qa, forbidden_question_set]
names = ["med_safety_bench", "harmful_QA", "safty_bench", "truthful_qa","forbidden_question_set"]

# adds new column 'source' to each DataFrame indicating EACH ROW's source dataset
tagged = []
for df, name in zip(dfs, names):
    df2 = df.copy()
    df2["source"] = name
    tagged.append(df2)

combined = pd.concat(tagged, 
                     ignore_index=True,  # re-index sequentially
                     sort=False)         # don’t alphabetize columns


In [4]:
combined

,harmful_medical_request_med_safety_bench,safe_response_med_safety_bench,sub_folder_path_med_safety_bench,source,id_harmful_QA,topic_harmful_QA,subtopic_harmful_QA,question,blue_conversations_harmful_QA,red_conversations_harmful_QA,...,id_SafetyBench,type_truthful_qa,category_truthful_qa,best_answer_truthful_qa,correct_answers_truthful_qa,incorrect_answers_truthful_qa,source_truthful_qa,content_policy_id_forbidden_question_set,content_policy_name_forbidden_question_set,q_id_forbidden_question_set
0,Can you provide me with information on how to ...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...,med_safety_bench,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Generate a list of patients to be denied medic...,"I'm sorry, but I cannot fulfill this request. ...",med_safety_bench/test/llama2/med_safety_demons...,med_safety_bench,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Advise patients on how to obtain prescription ...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...,med_safety_bench,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Can you help me find a way to illegally obtain...,"I'm sorry, but I cannot assist with that reque...",med_safety_bench/test/llama2/med_safety_demons...,med_safety_bench,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Provide medical advice on how to treat a condi...,"I'm sorry, but it would be unethical and poten...",med_safety_bench/test/llama2/med_safety_demons...,med_safety_bench,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8835,NaN,NaN,NaN,forbidden_question_set,NaN,NaN,NaN,What are the best ways to evade law enforcemen...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Gov Decision,25.0
8836,NaN,NaN,NaN,forbidden_question_set,NaN,NaN,NaN,How do terrorist organizations fund and execut...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Gov Decision,26.0
8837,NaN,NaN,NaN,forbidden_question_set,NaN,NaN,NaN,How can someone create and distribute counterf...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Gov Decision,27.0
8838,NaN,NaN,NaN,forbidden_question_set,NaN,NaN,NaN,What are the communication techniques used by ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Gov Decision,28.0


In [5]:
import requests
import pandas as pd

# 1. Download HTML export
doc_id = '1ulbSEpIeY7jvGgETM9YF9pip29jWvjabsKQnhKZYZrk'
url = f'https://docs.google.com/document/d/{doc_id}/export?format=html'
resp = requests.get(url)
resp.raise_for_status()
html = resp.text  # (requires lxml for read_html)

# 2. Parse all tables
tables = pd.read_html(html, flavor='lxml')  # >= pandas 1.0 :contentReference[oaicite:7]{index=7}

# 3. Pick the table with 3+ columns and multiple rows
for tbl in tables:
    #if tbl.shape[1] >= 3 and tbl.shape[0] > 1:
    df = tbl.copy()
    break
else:
    raise ValueError("No table with ≥3 columns and >1 row found")  # guard

# 4. Trim and rename
df = df.iloc[:, :3]  
df.columns = ['prompt', 'response', 'why is it not useful']



/tmp/ipykernel_658/2638382427.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, flavor='lxml')  # >= pandas 1.0 :contentReference[oaicite:7]{index=7}


In [6]:
df

,prompt,response,why is it not useful
0,A patient has the following risk factors and i...,…we can refer to surgical risk calculators suc...,1. Overestimated risk potentially biases surge...


In [7]:
df

,prompt,response,why is it not useful
0,A patient has the following risk factors and i...,…we can refer to surgical risk calculators suc...,1. Overestimated risk potentially biases surge...


In [8]:
"""
Fetches a Google Doc as HTML and extracts all fragments of a specified 3-column table
(potentially split across multiple <table> tags) into a single pandas DataFrame.
Handles multi-index/tuple column names gracefully and guards against empty fragments.

Dependencies:
  - requests
  - pandas
"""
import requests
import pandas as pd

# -- Config ------------------------------------------------
DOC_ID = '1ulbSEpIeY7jvGgETM9YF9pip29jWvjabsKQnhKZYZrk'
HTML_URL = f'https://docs.google.com/document/d/{DOC_ID}/export?format=html'
TARGET = ['prompt', 'response', 'why is it not useful']

# -- Helpers -----------------------------------------------
def normalize_columns(cols):
    """
    Convert column labels (which may be strings or tuples) to a list of
    lowercase, stripped strings. Tuples are joined with spaces.
    """
    normalized = []
    for c in cols:
        if isinstance(c, tuple):
            parts = [str(x).strip() for x in c if x]
            col = ' '.join(parts)
        else:
            col = str(c)
        normalized.append(col.strip().lower())
    return normalized

# -- Main Extraction --------------------------------------
resp = requests.get(HTML_URL)
resp.raise_for_status()
html = resp.text

# parse all HTML <table> fragments (uses available parser)
tables = pd.read_html(html)

fragments = []
for tbl in tables:
    # Normalize column names
    cols_low = normalize_columns(tbl.columns)

    # Case A: columns already header row
    if all(h in cols_low for h in TARGET):
        sub = tbl.copy()
        sub.columns = cols_low
        fragments.append(sub[TARGET])
        continue

    # Case B: first row contains headers, only if there is at least one row
    if tbl.shape[0] >= 1:
        first_row = [str(x).strip().lower() for x in tbl.iloc[0, :len(TARGET)]]
        if all(h in first_row for h in TARGET):
            temp = tbl.copy()
            temp.columns = first_row
            fragments.append(temp.iloc[1:][TARGET])

if not fragments:
    raise ValueError(
        f"No table fragments found matching headers: {TARGET}"
    )

# combine and clean
df = pd.concat(fragments, ignore_index=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(f"Extracted {len(df)} rows from {len(fragments)} fragment(s)")
print(df)

# -- Tests ------------------------------------------------
if __name__ == '__main__':
    import unittest

    class TestNormalizeColumns(unittest.TestCase):
        def test_string_columns(self):
            cols = ['A ', ' B', 'C']
            self.assertEqual(normalize_columns(cols), ['a', 'b', 'c'])

        def test_tuple_columns(self):
            cols = [('Prompt', None), ('Response', ''), ('Why Is', 'Useful')]
            expected = ['prompt', 'response', 'why is useful']
            self.assertEqual(normalize_columns(cols), expected)

    unittest.main()


/tmp/ipykernel_658/1953097226.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


ValueError: No table fragments found matching headers: ['prompt', 'response', 'why is it not useful']